In [38]:
%pylab inline
import time

Populating the interactive namespace from numpy and matplotlib


In [48]:
gr=1.0/5.0
gp=1.0/30.0
kr=1.0/1.0
kp=50.0
k=100

tiempo=1.0/gp
numero_de_celulas=250

p=zeros((numero_de_celulas,tiempo*10000))
r=zeros((numero_de_celulas,tiempo*10000))
tau=zeros((numero_de_celulas,tiempo*10000))

La única diferencia entre esta implementación y la que se usa anteriormente, es que se cambia la definicion de de kt y krkt por las siguientes:





->kt=kp*a+kr/(1+(b*b)/(k*k))+gp*b+gr*a






->krkt=(kr/(1+(b*b)/(k*k)))/kt

In [49]:
tamano=10000
for j in range (0,numero_de_celulas):
    p0=zeros(tiempo*10000)
    r0=zeros(tiempo*10000)
    r0[0]=100.0
    p0[0]=100.0
    t=0.0
    i=0.0
    while t<tiempo: 
        aleatorio=random.random()  
        a=r0[i]
        b=p0[i]  
        kt=kp*a+kr/(1+(b/k)**-2)+gp*b+gr*a
        kpkt=kp*a/kt
        gpkt=gp*b/kt
        grkt=gr*a/kt
        krkt=(kr/(1+(b/k)**-2))/kt
        if(aleatorio>=0)&(aleatorio<krkt):
            r0[i+1]=(a+1.0)
            p0[i+1]=(b)      
        elif(aleatorio<kpkt+krkt)&(aleatorio>=krkt) &(a!=0) :
            p0[i+1]=(b+1.0) 
            r0[i+1]=(a)
        elif(aleatorio<kpkt+krkt+gpkt)&(aleatorio>=kpkt+krkt) & (b!=0):
            p0[i+1]=(b-1.0) 
            r0[i+1]=(a)
        elif(aleatorio<1) & (aleatorio>=kpkt+krkt+gpkt) & (a!=0):
            r0[i+1]=(a-1.0)
            p0[i+1]=(b)
        c=random.exponential(1.0/kt)
        t=t+sum(c)
        tau[j][i+1]=t
        i=1+i
    if(tamano>i):
        tamano=i
    p[j]=p0
    r[j]=r0


KeyboardInterrupt: 

In [50]:
a=tamano
pfinal=zeros((numero_de_celulas,a+1))
rfinal=zeros((numero_de_celulas,a+1))
tau_final=zeros((numero_de_celulas,a+1))
for i in range (0,numero_de_celulas):
    pfinal[i]=(p[i])[0:a+1]
    rfinal[i]=(r[i])[0:a+1]
    tau_final[i]=(tau[i])[0:a+1]

In [ ]:
particion=tamano*10
dt=tamano/particion
print dt
p2=zeros((numero_de_celulas,particion))
r2=zeros((numero_de_celulas,particion))

for i in range (0,numero_de_celulas):
    for j in range (0, int(particion)):
        indice = (abs(tau_final[i]-j*dt)).argmin()
        p2[i][j]=pfinal[i][indice]
        r2[i][j]=rfinal[i][indice]
        
            

In [ ]:
a=0
b=0
for i in range(0, len(p2)):
    a=a+p2[i]
    b=b+r2[i]
a=a/len(p2)
b=b/len(r2)

In [ ]:
fig=plot(a)
title("Algoritmo de Gillespie para proceso con retroaliementacion para las proteinas",size=9)
ylabel('Poblaciones')
xlim(-100,3500)
xlabel('tiempo (micro-segundos)')
plt.savefig('Simulacion gillespie retroalimentada proteina.pdf',dpi=350)

In [ ]:
ylabel('Poblaciones')
xlabel('tiempo (micro-segundos)')
fig=plot(b)
xlim(-100,3500)
title("Algoritmo de Gillespie para proceso con retroaliementacion para ARN",size=8)
plt.savefig('Simulacion gillespie retroalimentada ARN.pdf',dpi=350)

In [ ]:
c=len(a)-1
distribucionp=empty(0)
distribucionr=empty(0)

for i in range (0,250):
    distribucionp=append(distribucionp,[p2[i][c]])
    distribucionr=append(distribucionr,[r2[i][c]])

    

Abajo de cada grafica se distribucion de encuentra primero el ruido y luego el promedio

In [ ]:
fig=count, bins, ignored = plt.hist(distribucionp, 50,normed=True)
title("Distribucion de proteinas con retroalimentacion-metodo Gillespie",size=9)
ylabel('Prbabilidad')
xlabel('Poblacion por celula')
plt.savefig('Distribucion de proteinas retroalimetada.pdf',dpi=350)
a= std(distribucionp)
b= mean(distribucionp)
ruido=a/b
print ruido
print b

In [ ]:
fig=count, bins, ignored = plt.hist(distribucionr, 10,normed=True)
title("Distribucion de ARN mensajero con retroalimentacion-metodo Gillespie",size=9)
ylabel('Prbabilidad')
xlabel('Poblacion por celula')
xlim(-1,4)
ylim(0,2)
plt.savefig('Distribucion de ARN mensajero retroalimetada.pdf',dpi=350)
c= std(distribucionr)
d = mean(distribucionr)
ruido=c/d
print ruido
print d